In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
directory = "../input/home-depot-product-search-relevance"
pickle_directory = "../input/btl-home-depot-product-search-relevance/"

# Importing libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib
import pickle
import keras as kr
from keras.preprocessing import text, sequence
from keras import layers, models, optimizers
from keras.layers import *
import xgboost
import gensim
from time import time
from gensim.models import KeyedVectors
from gensim.utils import simple_preprocess, tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
import spacy
import re
import math 
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder

# Data exploration

In [ ]:
train = pd.read_csv(directory + "/train.csv.zip", encoding = "ISO-8859-1", index_col= "id")
print(train.shape)
train.head(5)

In [ ]:
test = pd.read_csv(directory + "/test.csv.zip", encoding = "ISO-8859-1", index_col= "id")
print(test.shape)
test.head(5)

In [ ]:
attributes=pd.read_csv(directory + "/attributes.csv.zip")
print(attributes.shape)
attributes.head(5)

In [ ]:
hight_relevance = train.loc[train["relevance"] == 3]
hight_relevance[['product_title', 'search_term']].head()

Join table 'product_descriptions' and 'train', 'test'

In [ ]:
product_descriptions = pd.read_csv('../input/home-depot-product-search-relevance/product_descriptions.csv.zip')
train = pd.merge(train, product_descriptions, on='product_uid')
test = pd.merge(test, product_descriptions, on='product_uid')

In [ ]:
train.head()

In [ ]:
train.iloc[0]['search_term'] + '/' + train.iloc[0]['product_title'] + '/'+ train.iloc[0]['product_description']

## Plotting

In [ ]:
distribution = train['relevance'].value_counts().sort_index()
plt.figure(figsize=(6, 4))
plt.ylabel('count')
plt.xlabel('value of relevance')
distribution.plot(kind = 'bar')
plt.show()

In [ ]:
distribution_train = train.search_term.str.split().apply(len).value_counts().sort_index()
distribution_test = test.search_term.str.split().apply(len).value_counts().sort_index()
fig, (ax1, ax2) = plt.subplots(2, sharex=True)
fig.suptitle('Number of word in search term of train and test set')

distribution_train.plot(kind = 'bar', sharex = 1, figsize = (7,5), ax = ax1, title = 'train')
distribution_test.plot(kind = 'bar', colormap = 'jet', figsize = (7, 5), ax = ax2, title = 'test')

In [ ]:
distribution_train = train.product_description.str.split().apply(len).divmod(10)[0].value_counts().nlargest(40).sort_index()
distribution_test = test.product_description.str.split().apply(len).divmod(10)[0].value_counts().nlargest(40).sort_index()
fig, (ax1, ax2) = plt.subplots(2, sharex=True)
fig.suptitle('Number of word in product description of train and test set')

distribution_train.plot(kind = 'bar', sharex = 1, figsize = (10,5), ax = ax1, title = 'train')
distribution_test.plot(kind = 'bar', colormap = 'jet', figsize = (10, 5), ax = ax2, title = 'test')

# Data Pre-processing

Extract brand name and material information from attributes file

In [ ]:
print(len(train),' ',len(test))
test.tail()

In [ ]:
brand_attributes = attributes[attributes.name == "MFG Brand Name"][["product_uid", "value"]].rename(columns={"value": "brand"})
material_attributes = attributes[attributes.name == "Material"][["product_uid", "value"]].rename(columns={"value": "material"})
brand_attributes.drop_duplicates(subset=['product_uid'], inplace = True)
material_attributes.drop_duplicates(subset=['product_uid'], inplace = True)
train = pd.merge(train, brand_attributes, on='product_uid',  how='left').fillna('')
test = pd.merge(test, brand_attributes, on='product_uid',  how='left').fillna('')
train = pd.merge(train, material_attributes, on='product_uid',  how='left').fillna('')
test = pd.merge(test, material_attributes, on='product_uid',  how='left').fillna('')

In [ ]:
print(len(train),' ',len(test))
test.tail()

Open source : https://gist.github.com/susanli2016/b83d148de7394821509bd5172d2c96d3

## Processing the description

In [ ]:
strNum = {'zero': 0, 'one': 1, 'two': 2, 'three': 3, 'four': 4, 'five': 5, 'six': 6, 'seven': 7, 'eight': 8, 'nine': 9}
def str_stem(s): 
    if isinstance(s, str):
        #fix strong.Bullet strongBullet
        s = re.sub(r"(\w)\.([A-Z])", r"\1 \2", s)
        s = re.sub(r"([a-z])([A-Z])", r"\1 \2", s)
        s = re.sub(r"([0-9])( *)\.( *)([0-9])", r"\1.\4", s)
        #stem unit of measurement
        s = re.sub(r"([0-9]+)( *)(inches|inch|in|')\.?", r"\1in. ", s)
        s = re.sub(r"([0-9]+)( *)(foot|feet|ft|'')\.?", r"\1ft. ", s)
        s = re.sub(r"([0-9]+)( *)(pounds|pound|lbs|lb)\.?", r"\1lb. ", s)
        s = re.sub(r"([0-9]+)( *)(square|sq) ?\.?(feet|foot|ft)\.?", r"\1sq.ft. ", s)
        s = re.sub(r"([0-9]+)( *)(cubic|cu) ?\.?(feet|foot|ft)\.?", r"\1cu.ft. ", s)
        s = re.sub(r"([0-9]+)( *)(gallons|gallon|gal)\.?", r"\1gal. ", s)
        s = re.sub(r"([0-9]+)( *)(ounces|ounce|oz)\.?", r"\1oz. ", s)
        s = re.sub(r"([0-9]+)( *)(centimeters|cm)\.?", r"\1cm. ", s)
        s = re.sub(r"([0-9]+)( *)(milimeters|mm)\.?", r"\1mm. ", s)
        s = re.sub(r"([0-9]+)( *)(°|degrees|degree)\.?", r"\1 deg. ", s)
        s = re.sub(r"([0-9]+)( *)(v|volts|volt)\.?", r"\1 volt. ", s)
        s = re.sub(r"([0-9]+)( *)(wattage|watts|watt)\.?", r"\1 watt. ", s)
        s = re.sub(r"([0-9]+)( *)(amperes|ampere|amps|amp)\.?", r"\1 amp. ", s)
        s = re.sub(r"([0-9]+)( *)(qquart|quart)\.?", r"\1 qt. ", s)
        s = re.sub(r"([0-9]+)( *)(hours|hour|hrs.)\.?", r"\1 hr ", s)
        s = re.sub(r"([0-9]+)( *)(gallons per minute|gallon per minute|gal per minute|gallons/min.|gallons/min)\.?", r"\1 gal. per min. ", s)
        s = re.sub(r"([0-9]+)( *)(gallons per hour|gallon per hour|gal per hour|gallons/hour|gallons/hr)\.?", r"\1 gal. per hr ", s)
        # Deal with special characters
        s = s.replace("$"," ")
        s = s.replace("?"," ")
        s = s.replace("&nbsp;"," ")
        s = s.replace("&amp;","&")
        s = s.replace("&#39;","'")
        s = s.replace("/>/Agt/>","")
        s = s.replace("</a<gt/","")
        s = s.replace("gt/>","")
        s = s.replace("/>","")
        s = s.replace("<br","")
        s = s.replace("<.+?>","")
        s = s.replace("[ &<>)(_,;:!?\+^~@#\$]+"," ")
        s = s.replace("'s\\b","")
        s = s.replace("[']+","")
        s = s.replace("[\"]+","")
        s = s.replace("-"," ")
        s = s.replace("+"," ")
        s = s.replace("..",".")
        s = s.replace(",","") #could be number / segment later
        s = s.replace("-"," ")
        s = s.replace("//","/")
        s = s.replace("/"," ")
        s = s.replace("..",".")
        s = s.replace(" / "," ")
        s = s.replace(" \\ "," ")
        s = s.replace("."," . ")
        s = s.replace("  "," ")
        s = re.sub(r"(^\.|/)", r"", s)
        s = re.sub(r"(\.|/)$", r"", s)
        s = re.sub(r"([0-9])([a-z])", r"\1 \2", s)
        s = re.sub(r"([a-z])([0-9])", r"\1 \2", s)
        s = re.sub(r"([a-z])( *)\.( *)([a-z])", r"\1 \4", s)
        s = re.sub(r"([a-z])( *)/( *)([a-z])", r"\1 \4", s)
        s = s.replace("*"," xbi ")
        s = s.replace(" by "," xbi ")

        s = (" ").join([str(strNum[z]) if z in strNum else z for z in s.split(" ")])
        s = " ".join([re.sub('[^A-Za-z0-9-./]', ' ', word) for word in s.lower().split()])
        return s
    else:
        return "null"

### Spelling correctors

In [ ]:
#TODO : test SymSpell
!pip install pyspellchecker

In [ ]:
from spellchecker import SpellChecker
spell = SpellChecker()

### Remove stop words, tokenization, lemmatization. 

In [ ]:
stop_words = set(stopwords.words('english')) 
wordnet_lemmatizer = WordNetLemmatizer()
gensim.utils.tokenize
def preprocessing(doc, correction = 0):
    words = []
    doc  = str_stem(doc)
    for word in doc.split(' '):
        if word not in stop_words:
            if correction == 1:
                if len(spell.unknown([word])):
                    word = spell.correction(word)
            word1 = wordnet_lemmatizer.lemmatize(word, pos = "n")
            word2 = wordnet_lemmatizer.lemmatize(word1, pos = "v")
            word3 = wordnet_lemmatizer.lemmatize(word2, pos = ("a"))
            words.append(word3)
    return ' '.join(words)

In [ ]:
%%time

train.search_term = [preprocessing(i, correction = 1) for i in train.search_term]
train.product_description = [preprocessing(i) for i in train.product_description]
train.brand = [preprocessing(i) for i in train.brand]
train.material = [preprocessing(i) for i in train.material]
train.product_title = [preprocessing(i) for i in train.product_title]

test.search_term = [preprocessing(i, correction = 1) for i in test.search_term]
test.product_description = [preprocessing(i) for i in test.product_description]
test.brand = [preprocessing(i) for i in test.brand]
test.material = [preprocessing(i) for i in test.material]
test.product_title = [preprocessing(i) for i in test.product_title]

# train = pd.read_pickle(pickle_directory + "proccessed_train")
# test = pd.read_pickle(pickle_directory + "proccessed_test")

train.to_pickle("proccessed_train")
test.to_pickle("proccessed_test")

# Feature engineering

In [ ]:
features = pd.concat((train, test), axis=0, ignore_index=True)
print(str(len(train)) + ' ' + str(len(test)) + ' ' + str(len(features)))
del train
del test

In [ ]:
features['brand'] = features['brand'].replace(np.nan, '', regex=True)
features['material'] = features['material'].replace(np.nan, '', regex=True)

## String matching feature 

In [ ]:
from fuzzywuzzy import fuzz
#with description
features["partial_ratio_desc"] = [fuzz.partial_ratio(features["search_term"][i], features["product_description"][i]) for i in range(0, len(features))]
features["token_set_ratio_desc"] = [fuzz.token_set_ratio(features["search_term"][i], features["product_description"][i]) for i in range(0, len(features))]
features["token_sort_ratio_desc"] = [fuzz.token_sort_ratio(features["search_term"][i], features["product_description"][i]) for i in range(0, len(features))]
#with tittle
features["partial_ratio_tittle"] = [fuzz.partial_ratio(features["search_term"][i], features["product_title"][i]) for i in range(0, len(features))]
features["token_set_ratio_tittle"] = [fuzz.token_set_ratio(features["search_term"][i], features["product_title"][i]) for i in range(0, len(features))]
features["token_sort_ratio_tittle"] = [fuzz.token_sort_ratio(features["search_term"][i], features["product_title"][i]) for i in range(0, len(features))]
#with brand name
features["partial_ratio_brand"] = [fuzz.partial_ratio(features["search_term"][i], features["brand"][i]) for i in range(0, len(features))]
features["token_set_ratio_brand"] = [fuzz.token_set_ratio(features["search_term"][i], features["brand"][i]) for i in range(0, len(features))]
features["token_sort_ratio_brand"] = [fuzz.token_sort_ratio(features["search_term"][i], features["brand"][i]) for i in range(0, len(features))]
#with material
features["partial_ratio_material"] = [fuzz.partial_ratio(features["search_term"][i], features["material"][i]) for i in range(0, len(features))]
features["token_set_ratio_material"] = [fuzz.token_set_ratio(features["search_term"][i], features["material"][i]) for i in range(0, len(features))]
features["token_sort_ratio_material"] = [fuzz.token_sort_ratio(features["search_term"][i], features["material"][i]) for i in range(0, len(features))]

## String similarity feature

In [ ]:
pip install textdistance

In [ ]:
import textdistance
#with description
features["jaccard_similar_desc"] = [textdistance.jaccard(features["search_term"][i], features["product_description"][i]) for i in range(0, len(features))]
features["levenshtein_similarz_desc"] = [textdistance.levenshtein(features["search_term"][i], features["product_description"][i]) for i in range(0, len(features))]
features["mra_similar_desc"] = [textdistance.mra(features["search_term"][i], features["product_description"][i]) for i in range(0, len(features))]
#with title
features["jaccard_similar_title"] = [textdistance.jaccard(features["search_term"][i], features["product_title"][i]) for i in range(0, len(features))]
features["levenshtein_similarz_title"] = [textdistance.levenshtein(features["search_term"][i], features["product_title"][i]) for i in range(0, len(features))]
features["mra_similar_title"] = [textdistance.mra(features["search_term"][i], features["product_title"][i]) for i in range(0, len(features))]
#with brand
features["jaccard_similar_brand"] = [textdistance.jaccard(features["search_term"][i], features["brand"][i]) for i in range(0, len(features))]
features["levenshtein_similarz_brand"] = [textdistance.levenshtein(features["search_term"][i], features["brand"][i]) for i in range(0, len(features))]
features["mra_similar_brand"] = [textdistance.mra(features["search_term"][i], features["brand"][i]) for i in range(0, len(features))]
#with material
features["jaccard_similar_material"] = [textdistance.jaccard(features["search_term"][i], features["material"][i]) for i in range(0, len(features))]
features["levenshtein_similarz_material"] = [textdistance.levenshtein(features["search_term"][i], features["material"][i]) for i in range(0, len(features))]
features["mra_similar_material"] = [textdistance.mra(features["search_term"][i], features["material"][i]) for i in range(0, len(features))]

## Word embedding feature

In [ ]:
from nltk.corpus import brown
embed_model = gensim.models.Word2Vec(brown.sents())
embed_model.save('brown.embedding')
model = gensim.models.Word2Vec.load('brown.embedding')

In [ ]:
print(model.wv.similarity("humanity", "angel"), ' ', model.wv.similarity("humanity", "evil"))
print(model.wv.similarity("cat", "alien"), ' ', model.wv.similarity("cat", "human"))


In [ ]:
%%time
def embeding_similarity_calculator(s, t, i):
    _sum = 0
    avg = 0
    if len(s.split()) == 0 :
        return 0
    for s_word in s.split():
        _max = 0
        for t_word in t.split():
            if ((s_word in model.wv) and (t_word in model.wv)):
                _max = max(_max, model.wv.similarity(s_word, t_word))
        _sum += _max
    avg = _sum/ len(s.split())
    return avg
features["word_ebed_similarity"] = [embeding_similarity_calculator(features["search_term"][i], features["product_description"][i], i) for i in range(0, len(features))]

## TF-IDF similarity measure feature at char level

In [ ]:

tfidf_vect = TfidfVectorizer(analyzer='char_wb', ngram_range = (3,3), max_features = 1500)
tfidf_des = tfidf_vect.fit_transform(features.product_description).toarray()
tfidf_search = tfidf_vect.transform(features.search_term).toarray()

# infile = open(pickle_directory + "tfidf_des",'rb')
# tfidf_des = pickle.load(infile)
# infile = open(pickle_directory + "tfidf_search",'rb')
# tfidf_search = pickle.load(infile)

outfile = open("tfidf_des",'wb')
pickle.dump(tfidf_des, outfile)
outfile = open("tfidf_search",'wb')
pickle.dump(tfidf_search, outfile)

In [ ]:
from scipy.spatial import distance
features["tfidf_cosine_distance"] = [distance.cosine(tfidf_search[i], tfidf_des[i]) for i in range(0, len(tfidf_des))]

In [ ]:
# infile = open("../input/btl-home-depot-product-search-relevance/features_18_4",'rb')
# features = pickle.load(infile)
outfile = open("features_18_4",'wb')
pickle.dump(features, outfile)

# Model

In [ ]:
features = pd.DataFrame(features).fillna(0)
df_train = features.iloc[:74067]
df_test = features.iloc[74067:]
y_train = df_train['relevance']
df_train = df_train.drop(columns=['product_title','product_description','brand','material','search_term','relevance'])
df_test = df_test.drop(columns=['product_title','product_description','brand','material','search_term', 'relevance'])

In [ ]:
print(len(df_train), ' ',len(df_test))

## Metric to evaluate models : MAE, RMSE

In [ ]:
df_train = df_train.sort_values('product_uid')

In [ ]:
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error

def modelEvaluate(model, X_train, y_train, x_val, y_val, label_encode = None):
    model.fit(X_train, y_train)
    pred = model.predict(x_val)
    _pred = pred
    _y_val = y_val
    if label_encode is not None:
        _y_val = label_encode.inverse_transform(_y_val)
        _pred = label_encode.inverse_transform(_pred)
    mae, mse = (mean_absolute_error(_pred, _y_val),mean_squared_error(_pred, _y_val))
    return (mae, mse)

In [ ]:
from sklearn.model_selection import KFold

def crossValidate(model, label_encoder = None):
    mae_score = []
    mse_score = []
    kf = KFold(n_splits=4)
    kf.get_n_splits(df_train)
    for train_index, test_index in kf.split(df_train):
        X, X_test = df_train.iloc[train_index], df_train.iloc[test_index]
        y, y_test = y_train.iloc[train_index], y_train.iloc[test_index]
        (mae, mse) = modelEvaluate(model, X, y, X_test, y_test, label_encode = label_encoder)
        mae_score.append(mae)
        mse_score.append(mse)
    return [sum(mae_score)/len(mae_score), math.sqrt(sum(mse_score)/len(mse_score))]

## Regressor

In [ ]:
%%time
from xgboost import XGBRegressor
from sklearn.svm import SVR
from sklearn.ensemble import ExtraTreesRegressor, GradientBoostingRegressor, RandomForestRegressor
from sklearn import linear_model

result = []
result.append(('RandomForest',crossValidate(RandomForestRegressor(n_estimators=700, max_depth=6, random_state=42))))
result.append(('XGBoost',crossValidate(XGBRegressor(colsample_bytree=0.4,       
                 learning_rate=0.1,
                 max_depth=6,
                 n_estimators=700,                                                                    
                 reg_alpha=0.075,
                 reg_lambda=0.045,
                 subsample=0.6,
                 seed=42))))
result.append(('GradientBoosting',crossValidate(GradientBoostingRegressor(n_estimators=700, max_depth=6, random_state=42))))
result.append(('ETR',crossValidate(ExtraTreesRegressor(n_estimators=700, max_depth=5, random_state=42))))
result.append(('Lasso',crossValidate(linear_model.Lasso(alpha=0.05))))
result.append(('ElasticNet',crossValidate(linear_model.ElasticNet(alpha=0.05))))
result.append(('Ridge',crossValidate(linear_model.Ridge(alpha=0.05))))

In [ ]:
result

In [ ]:
%%time
from sklearn import linear_model
from sklearn.ensemble import StackingRegressor
from sklearn.linear_model import BayesianRidge
estimators = [  ('RandomForest', RandomForestRegressor(n_estimators=700, max_depth=6, random_state=42)),
                ('XGBoost', XGBRegressor(colsample_bytree=0.4,learning_rate=0.1,max_depth=6,n_estimators=700,reg_alpha=0.075,reg_lambda=0.045,subsample=0.6,seed=42)),                                                                    
                ('Lasso',linear_model.Lasso(alpha=0.05)),
                ('ElasticNet',linear_model.ElasticNet(alpha=0.05)),
                ('Ridge',linear_model.Ridge(alpha=0.05))
             ]
stacked_reg = StackingRegressor(
        estimators=estimators,
        final_estimator=BayesianRidge()
    )
result.append(('StackedBayesianRidge',crossValidate(stacked_reg)))

In [ ]:
labels = []
mae = []
rmse = []
for _name, _score in result:
    labels.append(_name)
    mae.append(_score[0])
    rmse.append(_score[1])

#Plotting
x = np.arange(len(labels))  # the label locations
width = 0.3  # the width of the bars

fig, ax = plt.subplots(figsize=(15,6))
rects1 = ax.bar(x - width/2, mae, width, label='MAE')
rects2 = ax.bar(x + width/2, rmse, width, label='RMSE')

ax.set_ylabel('Scores')
ax.set_title('Scores by models')
ax.set_xticks(x)
ax.set_xticklabels(labels)
ax.legend()

ax.bar_label(rects1, padding=3)
ax.bar_label(rects2, padding=3)

fig.tight_layout()

plt.show()

In [ ]:
type(y_train)

## Class weight +  classifier

In [ ]:
%%time
le = LabelEncoder()
y_train = pd.Series(data = le.fit_transform(y_train))
y_train = y_train.replace({1:2, 3:4, 5:6})

from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.linear_model import Perceptron
result_classifier = []
result_classifier.append(('ETR_c',crossValidate(ExtraTreesClassifier(n_estimators=700,max_depth = 6, class_weight = 'balanced_subsample', random_state=42),le)))
result_classifier.append(('RF_c',crossValidate(RandomForestClassifier(n_estimators=700,max_depth = 6, class_weight = 'balanced_subsample', random_state=42),le)))
result_classifier.append(('PA_c',crossValidate(PassiveAggressiveClassifier(max_iter = 2000, early_stopping = True, class_weight = 'balanced'),le)))
result_classifier.append(('Perceptron_c',crossValidate(Perceptron(max_iter = 2000,penalty = 'l2', class_weight = 'balanced', early_stopping = True, random_state=42),le)))

In [ ]:
y_train.replace({1:2, 3:4, 5:6, 7:8, 9:10, 11:12}).value_counts()

In [ ]:
labels = []
mae = []
rmse = []
for _name, _score in result_classifier:
    labels.append(_name)
    mae.append(_score[0])
    rmse.append(_score[1])

#Plotting
x = np.arange(len(labels))  # the label locations
width = 0.3  # the width of the bars

fig, ax = plt.subplots(figsize=(15,6))
rects1 = ax.bar(x - width/2, mae, width, label='MAE')
rects2 = ax.bar(x + width/2, rmse, width, label='RMSE')

ax.set_ylabel('Scores')
ax.set_title('Scores by models')
ax.set_xticks(x)
ax.set_xticklabels(labels)
ax.legend()

ax.bar_label(rects1, padding=3)
ax.bar_label(rects2, padding=3)

fig.tight_layout()

plt.show()

In [ ]:
y_train = features.iloc[:74067]['relevance']

## Make submission

In [ ]:
stacked_reg.fit(df_train, y_train)
pred = stacked_reg.predict(df_test)
pred.shape

In [ ]:
submission =pd.read_csv(directory + "/sample_submission.csv.zip")
pred = [min(3,i) for i in pred]
pred = [max(1, i) for i in pred]
submission['relevance'] = pred
submission.to_csv('submission.csv', index=False)

## Feature importance

In [ ]:
rf = RandomForestRegressor(n_estimators=500, max_depth=5, random_state=42)
rf.fit(df_train, y_train)
fi = pd.DataFrame({'feature': df_train.columns, 'importance': rf.feature_importances_}).sort_values(by='importance', ascending=False)
fi = fi.reset_index()
fi.head(20)